In [14]:
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import cryptography.exceptions
import os
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import padding
import time


# Emitter
No enunciado era pedida uma comunação assíncrona entre uma entidade emissora e uma entidade recetora. O que o nosso grupo fez foi uma simulação local desta comunicação. 

A classe emitter tem 2 argumentos, uma private_key e uma shared_key. A private key será gerada tendo em conta os parametros acordados entre o emissor e o recetor enquanto que a shared_key será obtida através das chaves publicas associadas às chaves privadas do recetor e do emissor. A chave partilhada calculada nuncva vai ser enviada para nenhum lado e vai ser utilizada na autenticação HMAC. Desta maneira, é muito mais difícil para um atacante obter esta chave visto que precisa de uma chave privada e de uma chave publica.

Na função talk_to_you enviamos a mensagem. Usamos o método HMAC para autentificação e depois para evitar ataques de nonce geramos um valor aleatório

In [15]:
class Emitter:
    def __init__(self):
        self.private_key = None
        self.shared_key  = None

    def get_private_key(self, parameters):
        self.private_key = parameters.generate_private_key()

    def get_shared_key(self, receiver_public_key):
        key = self.private_key.exchange(receiver_public_key)
        # Derivar chave
        self.shared_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info=None,
        ).derive(key)

    def talk_to_you(self, message):
        h = hmac.HMAC(self.shared_key, hashes.SHA256())
        h.update(b"YOYOYO")
        signature = h.finalize()

        nonce = os.urandom(16)
        alg = AESGCM(self.shared_key)
        aad = os.urandom(16)
        cipher_text = alg.encrypt(nonce, message, aad)
        return (signature + nonce + aad + cipher_text)

A classe Receiver tem tal como a classe Emitter uma chave privada e uma chave partilhada que são obtidas de igual forma.

A maior distinção está na função talk_to_me onde ele pretende verificar se a mensagem que tem é de facto igual à mensagem enviada pelo emissor. Dos dados recebidos ele faz uma separação deles. Começa por verificar a autentiação com um método HMAC, e caso se verifique que de facto é a mesma mensagem, ele decifra a mensagem recebida e mostra-a ao utilizador.

In [16]:
class Receiver:
    def __init__(self):
        self.private_key = None
        self.shared_key  = None

    def get_private_key(self, parameters):
        self.private_key = parameters.generate_private_key()

    def get_shared_key(self, emitter_public_key):
        key = self.private_key.exchange(emitter_public_key)
        # Derivar chave
        self.shared_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info=None,
        ).derive(key)

    def talk_to_me(self, data):
        isValid = False
        signature = data[:32]
        nonce = data[32:48]
        aad = data[48:64]
        cipher_text = data[64:]
        verified = ""

        h = hmac.HMAC(self.shared_key, hashes.SHA256())
        h.update(b"YOYOYO")
        try: 
            h.verify(signature)
            verified = "Yo"
        except cryptography.exceptions.InvalidSignature:
            verified = "Not Yo"

        if verified == 'Yo':
            isValid = True

        if(isValid):
            alg = AESGCM(self.shared_key)
            clean_text = alg.decrypt(nonce, cipher_text, aad)
            print(clean_text)

Vamos agora simular a nossa comunicação assíncrona. Iniciamos o emissor e o recetor, e geramos parâmetros comuns aos dois que serão utilizados para obter as chaves privadas. Normalmente estes parâmetros seriam acordados entre as duas entidades, mas aqui para facilitar nmão temos essa discussão de parâmetros.

De seguida cada entidade gera a sua própria chave privada, chave esta que nunca deve ser comunicada a mais ninguém. Aquilo que é enviado é a chave pública associada à chave privada, esta sim pode ser partilhada de forma totalmente livre sem que seja preciso preocupar-nos se algúem a apanha. Tendo o emissor a sua chave privada e a chave publica do recetor, ele gera uma chave partilhada; do outro lado tendo o recetor a sua chave privada e a chave pública do emissor, a mesma chave partilhada é obtida.

Finalmente estão prontos para comunicar um com o outro. O que o grupo aqui quer fazer é dada uma certa mensagem, verificar que tanto o emissor e o recetor têm a mesma mensagem. Para isso fazemos uma autenticação com o método HMAC, com passos extra para evitar ataques nonce. No caso da mensagem ser igual dos dois lados é retornada a mensagem, caso contrário não há retorno. Contamos o tempo deste processo para comparar com o exercício 2.

In [17]:
start = time.time()

emitter  = Emitter()
receiver = Receiver()

message = b'MACACO CUSPIDEIRA BUFALO'

# Geramos parametros acordados entre emissor e recetor
parameters = dh.generate_parameters(generator=2, key_size=1024)

# Private keys
emitter.get_private_key(parameters)
receiver.get_private_key(parameters) 

# Shared Keys
emitter.get_shared_key(receiver.private_key.public_key())
receiver.get_shared_key(emitter.private_key.public_key())

# Autenticação 
data = emitter.talk_to_you(message)
receiver.talk_to_me(data)

end = time.time()
print(end - start)

TypeError: talk_to_me() takes 2 positional arguments but 3 were given